Take two gromacs trajectories of the same complex with two different initial conditions and produce a third 'averaged' trajectory from them. 

In [ ]:
import MDAnalysis as mda
import numpy as np

# Load the two trajectories
u1 = mda.Universe('topology_file1.pdb', 'trajectory_file1.xtc')
u2 = mda.Universe('topology_file2.pdb', 'trajectory_file2.xtc')

# Check that the two trajectories have the same number of frames and the same number of atoms
assert len(u1.trajectory) == len(u2.trajectory), "Trajectories must have the same number of frames"
assert u1.atoms.n_atoms == u2.atoms.n_atoms, "Trajectories must have the same number of atoms"

# Create a new Universe for the averaged trajectory
u_avg = mda.Universe.empty(u1.atoms.n_atoms, trajectory=True)
u_avg.add_TopologyAttr('name', u1.atoms.names)
u_avg.add_TopologyAttr('type', u1.atoms.types)
u_avg.add_TopologyAttr('resname', u1.atoms.resnames)
u_avg.add_TopologyAttr('resid', u1.atoms.resids)

# Iterate through each frame, compute the average, and append to the new trajectory
with mda.Writer('averaged_trajectory.xtc', n_atoms=u1.atoms.n_atoms) as W:
    for ts1, ts2 in zip(u1.trajectory, u2.trajectory):
        avg_positions = (ts1.positions + ts2.positions) / 2.0
        u_avg.atoms.positions = avg_positions
        W.write(u_avg.atoms)

print("Averaged trajectory saved as 'averaged_trajectory.xtc'")